Calculation of Kalman filter steps

I first show all the calculations I did and then I
will present the full requested table.

From the motion equation:

$$x_k = x_{k-1} + \epsilon_t \quad \textrm{with} \quad \epsilon_t=\mathcal{N}(0,2)$$

we can derive $r^2=2$.

Additionally, from the measurement equation:

$$z_k = x_{k} + \delta_t \quad \textrm{with} \quad \delta_t=\mathcal{N}(0,3)$$

we can derive $q^2=3$.

The initial belief is:

\begin{equation}
\mu_0 = 2, \Sigma_0 = 5
\end{equation}

First we start with the predict step:

\begin{equation}
\bar{\mu}_1 = \mu_0 = 2\\
\bar{\Sigma}_1 = \Sigma_0 + r^2 = 5 + 2 = 7
\end{equation}

Next we continue with the Kalman gain:

\begin{equation}
K_1 = \frac{\bar{\Sigma}_1}{\bar{\Sigma}_1 + \sigma_2}
\end{equation}


